In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2


import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import utils


In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Data

Data for this is from https://gmd.copernicus.org/articles/11/1133/2018/ CDRMIP data, where pi-CO2pulse is the 100GtC pulse and piControl is the control

### Notes on data
ACCESS: 10 years of pi control on the co2pulse run; take G 10 years in

GFDL: pi control run for 100 years before pulse, co2pulse run starts at 100 years in without any picontrol data

UKESM1: 10 years of pi control on the co2pulse run; take G 10 years in

MIROC: co2pulse starts 10 years after pi control, but doesn't include that in its data

NORESM2: picontrol starts in 1851 and co2pulse starts in 1850, both should start in 1851 according to documentation

CanESM5 (p2): picontrol is at 5601, but was branched at 5301; co2pulse was branched at 5401;line up by moving picontrol dt to start at 5301

In [3]:
model_run_pulse_dict = utils.model_run_pulse_dict
model_run_1pct_dict = utils.model_run_1pct_dict
model_run_control_dict = utils.model_run_esm_picontrol_dict


In [4]:
A = utils.A
ds_out = utils.ds_out

In [ ]:
# Define the options for our pulses

pulse_types = ['cdr','pulse']

In [5]:
for pulse_type in pulse_types:
    ds_control = {}
    ds_pulse = {}
    G = {}

    for m1 in model_run_pulse_dict.keys():
        if m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
            m2 = 'CANESM5_r1p2'
        else:
            m2 = m1
        ds_control[m1], ds_pulse[m1], G[m1] = utils.import_regrid_calc(f'cmip6_data/tas_Amon_{model_run_control_dict[m2]}', 
                                                     f'cmip6_data/tas_Amon_{model_run_pulse_dict[m1]}', 
                                                     ds_out, variable = 'tas', m = m1, pulse_type = pulse_type, pulse_size = 100, replace_xy = True)

    #take the annual means
    for m in ds_control.keys():

        times = ds_control[m].time.get_index('time')
        weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
        weights = xr.DataArray(weights, [('time', ds_control[m]['time'].values)]).astype('float')
        ds_control[m] =  (ds_control[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')



        times = ds_pulse[m].time.get_index('time')
        weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
        weights = xr.DataArray(weights, [('time', ds_pulse[m]['time'].values)]).astype('float')
        ds_pulse[m] =  (ds_pulse[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')


        ds_control[m]['year'] = ds_control[m]['year'] - ds_control[m]['year'][0]
        ds_pulse[m]['year'] = ds_pulse[m]['year'] - ds_pulse[m]['year'][0]
        
    control = xr.concat([ds_control[m] for m in ds_control.keys()], pd.Index([m for m in ds_control.keys()], name='model'))
    pulse = xr.concat([ds_pulse[m] for m in ds_pulse.keys()], pd.Index([m for m in ds_pulse.keys()], name='model'))
    
    ## test two ways of the polyfit: before and after finding G
    
    #polyfit the control and pulse before finding G
    poly_cont = control.polyfit('year', 4)
    control_polyfit = xr.polyval(control.year, poly_cont)['tas_polyfit_coefficients']

    poly_pulse = pulse.polyfit('year', 4)
    pulse_polyfit = xr.polyval(pulse.year, poly_pulse)['tas_polyfit_coefficients']

    G_poly_base_pulse = (pulse_polyfit - control_polyfit)/100
    
    #polyfit the G
    G_poly_normal_base_pulse = (pulse - control)/100
    
    Gpoly = G_poly_normal_base_pulse.polyfit('year', 4)
    G_poly_normal_base_pulse = xr.polyval(G_poly_normal_base_pulse.year, Gpoly)['tas_polyfit_coefficients']

    #combine the two
    G = xr.concat([G_poly_normal_base_pulse, G_poly_base_pulse], pd.Index(['normal_base_pulse', 'polyfit_base_pulse'], name = 'polyfit_type'))
    
    #save out
    G.to_netcdf(f'Outputs/G_poly_{pulse_type}_ds.nc4')


UKESM1_r1 UKESM1_r1


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


normal run
MIROC MIROC


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


normal run
NORESM2 NORESM2


...

normal run
CANESM5_r1p2 CANESM5_r2p2


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


normal run
CANESM5_r1p2 CANESM5_r3p2


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


normal run
